In [1]:
!python3 -m pip install gym
!python3 -m pip install tflearn
!python3 -m pip install python-opengl

  Could not find a version that satisfies the requirement python-opengl (from versions: )
No matching distribution found for python-opengl


In [0]:
import gym
import random
import numpy as np
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from statistics import mean, median
from collections import Counter


In [0]:
LR = 1e-3
env = gym.make('CartPole-v0')
env.max_episode_steps = None
env._max_episode_steps = None
env.reset()
goal_steps = 500 #how many step
score_requirement = 50
initial_game = 10000 #brute force every answer


In [0]:
def some_random_games_first():
    for episode in range(5):
        env.reset()
        for t in range(200):
            env.render() # if you want faster, don't render
            action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
            #observation have 4 values at same time
            #print(observation)
            if done:
                break
#some_random_games_first()

In [0]:
def initial_population():
    training_data = []
    scores = []
    accepted_scores = []
    for _ in range(initial_game):
        score =0
        game_memory = []
        prev_observation = []
        for _ in range(goal_steps):
            action = random.randrange(0,2)
            #action = 0 or 1 [left, right]
            observation, reward, done, info = env.step(action)
            
            # notice that the observation is returned FROM the action
            # so we'll store the previous observation here, pairing
            # the prev observation to the action we'll take.
            if len(prev_observation)>0:
                #game_memory save all obersation and action
                game_memory.append([prev_observation, action])
                #print('prev_observation:',prev_observation,' action:',action)
                
            prev_observation = observation
            score +=reward
            if done:
                break
        if score >= score_requirement:
            #accepted_scores just save score above score_requirement
            accepted_scores.append(score)
            #to change action '1'-->[0,1] or action '0' -->[1,0] for suit with numpy data style
            # we change to [0,1] because we also keep 0 or 1, but some game have more action, so we use [0,1]
            for data in game_memory:
                if data[1]== 1:
                    output = [0,1]
                elif data[1] == 0:
                    output = [1,0]
                #saving our traning data
                training_data.append([data[0],output])
        env.reset()
        #save all score values of each espiso, also counting value not enough for the requirement
        scores.append(score)
    #just in case you wanted to reference later
    training_data_save = np.array(training_data)
    np.save('./saved.npy', training_data_save)
    
    #some stats here,to further illustrate the neural network magic
    print('Average accepted score:', mean(accepted_scores))
    print('Median score for accepted scores:', median(accepted_scores))
    #counter is keep track how many value repeat
    print(Counter(accepted_scores))
    return training_data
#initial_population()

In [0]:
#training_data = initial_population()
#training_data
i=0
#print(training_data[0][0].reshape(-1,4,1))
#type(training_data)

Dữ liệu training và predict sẽ có dạng:
- [[[-0.04253177],[-0.21715409],[ 0.03958076],[ 0.28797414]]]
- training_data[0][0].reshape(-1,len(training_data[0][0]),1)


In [0]:
def neural_network_model(input_size):
    #1 in input_size mean just observation source for input 
    network = input_data(shape=[None, input_size, 1], name='input')
    #128 nodes for fisrt layer, each node have activation is relu for linear function
    network = fully_connected(network, 128, activation='relu')
    #0.8 is keep rate
    network = dropout(network, 0.8)
    #seconde layer
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
    model = tflearn.DNN(network, tensorboard_dir='log')
    #sometimes have out of memony, so just change number of layer
    return model


In [8]:
def train_model(training_data, model=False):
    #training_data have pype [[4 observation][0,1 or 1,0]]
    #[i[0] for i in training_data] mean all element in first place (4 observations)
    # X need np.array because training_data has type is list, can't not reshape -->should be convert to numpy.array type
    X = np.array([i[0] for i in training_data]).reshape(-1,len(training_data[0][0]),1)#---> X has shape: (totall number,4,1) -->the official shape for training
    #[i[0] for i in training_data] mean all elements in the secon place [0,1] or [1,0]
    y = [i[1] for i in training_data] #----> [0,1] or [1,0]
    #print(X.shape)
    #print(y[0])
    if not model:
        #run neural_network_model
        model = neural_network_model(input_size = len(X[0])) #len(X[0])=4
    #just run 1 time becasue model.fit can re-run
    model.fit({'input': X}, {'targets': y}, n_epoch=3, snapshot_step=500, show_metric=True, run_id='openai_learning')
    #n-epoch in simple case don't too high, <=95% accurate
    #snapshot_step mean how much step trained will print out the screen
    return model
training_data = initial_population()
model = train_model(training_data)

Training Step: 1040  | total loss: 0.66025 | time: 4.894s
| Adam | epoch: 003 | loss: 0.66025 - acc: 0.5916 -- iter: 22144/22207
Training Step: 1041  | total loss: 0.66547 | time: 4.906s
| Adam | epoch: 003 | loss: 0.66547 - acc: 0.5872 -- iter: 22207/22207
--


In [0]:
#model.save('hihi.model')

In [10]:
X=np.array([i[0] for i in training_data]).reshape(-1,len(training_data[0][0]),1)
len(X[1])

4

In [0]:
#model.load('hihi.model')

In [36]:
scores = []
choices = []
for each_game in range(10):
    score = 0
    game_memory = []
    prev_obs = []
    env.reset()
    for _ in range(1000):
        #env.render()

        if len(prev_obs)==0:
            action = random.randrange(0,2)
        else:
            action = np.argmax(model.predict(prev_obs.reshape(-1,len(prev_obs),1))[0])

        choices.append(action)
                
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        game_memory.append([new_observation, action])
        score+=reward
        if done:
          break

    scores.append(score)

print('Average Score:',sum(scores)/len(scores))
print('choice 1:{}  choice 0:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices)))


Average Score: 721.7
choice 1:0.49688236109186645  choice 0:0.5031176389081335


In [37]:
list(scores)
#max(scores)

[549.0, 1000.0, 1000.0, 1000.0, 1000.0, 362.0, 1000.0, 1000.0, 139.0, 167.0]

In [0]:
env._max_episode_seconds

In [35]:
t=model.predict([[[-0.04253177],[-0.21715409],[ 0.03958076],[ 0.28797414]]])
t

array([[0.49776545, 0.5022345 ]], dtype=float32)